<a href="https://colab.research.google.com/github/ZeinabHom/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RY01SAV7HZ8QO|          4|            0|          0|   N|                Y|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|                N|
|R149Q3B5L33NN5|          5|            0|          0|   N|                Y|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|                Y|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|                Y|
|R3L6FGKAW0EYFI|          1|            1|          1|   N|                N|
| RAO0QZH5VC6VI|          1|            0|          0|   N|                Y|
|R25IK0UAHWNB22|          3|            0|          0|   N|                Y|
|R2A9IHKZMTMAL1|          5|            1|          2|   N|                Y|
| R5XVKTHL6SITI|          5|            0|          0|   N|     

In [50]:
vine_vote_greater_20_df=vine_df.filter("total_votes>=20")
vine_vote_greater_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
|R1LV778IDGRJQ6|          1|            4|         23|   N|                Y|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R2LBEAGVJA7HK1|          3|            5|         48|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R3QLX5NODD1OJ4|          1|            9|         25|   N|                Y|
|R2SOOHE0XUQON1|          1|            2|         24|   N|     

In [51]:
vine_helpful_df=df.filter("helpful_votes/total_votes>=0.5")
vine_helpful_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    4417771|R3L6FGKAW0EYFI|B004N866SU|     414565179|Samsung 3D LED HD...|Home Entertainment|          1|            1|          1|   N|                N|Only lasts 3-4 ye...|I bought this TV ...| 2015-08-31|
|         US|   20691979|R2A9IHKZMTMAL1|B00QHLSKOE|     885228855|Matricom G-Box Q²...|Home Entertainment|      

In [52]:
vine_y_df=vine_helpful_df.filter("vine == 'Y'")
vine_y_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51288086|R3LLICBKRHLRJC|B00USEZ7W4|     202667718|Seiki SE32HYT  LE...|Home Entertainment|          4|           17|         22|   Y|                N|Great picture, a ...|Very nice, sharp ...| 2015-08-27|
|         US|   20151467| RJRUL44DZXFXU|B00X6AGWRQ|     467547785|Sony BDPS6500 - B...|Home Entertainment|      

In [53]:
vine_rating_df=vine_df.groupby("star_rating").count()
vine_rating_df.show()

+-----------+------+
|star_rating| count|
+-----------+------+
|          1| 99459|
|          3| 57167|
|          5|373988|
|          4|131508|
|          2| 43767|
+-----------+------+



In [29]:
vine_df.filter("star_rating ==5").count()

373988

In [44]:
vine_df.filter("(vine == 'N')").count()

703783

In [45]:
vine_df.filter("(vine == 'Y')").count()

2106

In [46]:
(vine_df.filter("star_rating ==5").count())/(vine_df.filter("(vine == 'N')").count())*100

53.13967515555221

In [33]:
(vine_df.filter("star_rating ==5").count())/(vine_df.filter("vine == 'Y'").count())*100

17758.21462488129

In [47]:
(vine_df.filter("star_rating ==5").count())/(vine_df.filter("(verified_purchase == 'N')").count())*100

205.54212099894477

In [48]:
(vine_df.filter("star_rating ==5").count())/(vine_df.filter("(verified_purchase == 'Y')").count())*100

71.38033771235854